In [1]:
#合并代码
import csv
import random
import math

#加载数据
def loadCsv(filename):
  lines = csv.reader(open(filename, "r"))
  dataset = list(lines)
  for i in range(len(dataset)):
    dataset[i] = [float(x) for x in dataset[i]]
  return dataset

#划分数据集
def splitDataset(dataset, splitRatio):
  trainSize = int(len(dataset) * splitRatio)
  trainSet = []
  copy = list(dataset)
  while len(trainSet) < trainSize:
    index = random.randrange(len(copy))
    trainSet.append(copy.pop(index))
  return [trainSet, copy]

#按类划分
def separateByClass(dataset):
  separated = {}
  for i in range(len(dataset)):
    vector = dataset[i]
    if (vector[0] not in separated):
      separated[vector[0]] = []
    separated[vector[0]].append(vector)
  return separated

#求均值方差
def mean(numbers):
  return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
  avg = mean(numbers)
  variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
  return math.sqrt(variance)

#提取数据集的特征
#zip函数将数据样本按照属性分组为一个个列表，然后可以对每个属性计算均值和标准差
def summarize(dataset):
  summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
  del summaries[0]
  return summaries

#按类提取特征
def summarizeByClass(dataset):
  separated = separateByClass(dataset)
  summaries = {}
  for classValue, instances in separated.items():
    summaries[classValue] = summarize(instances)
  return summaries

#计算高斯概率密度函数
def calculateProbability(x, mean, stdev):
  exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
  return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

#计算所属类的概率
def calculateClassProbabilities(summaries, inputVector):
  probabilities = {}
  for classValue, classSummaries in summaries.items():
    probabilities[classValue] = 1
    #此处是按classSummaries中的属性从头到尾与test的数据的属性进行比较，因为test的正式属性从第二个开始所以多重预测中的测试向量要从1开始
    for i in range(len(classSummaries)):
      mean, stdev = classSummaries[i]
      x = inputVector[i]
      probabilities[classValue] *= calculateProbability(x, mean, stdev)
  return probabilities

#单一预测
def predict(summaries, inputVector):
  probabilities = calculateClassProbabilities(summaries, inputVector)
  bestLabel, bestProb = None, -1
  for classValue, probability in probabilities.items():
    if bestLabel is None or probability > bestProb:
      bestProb = probability
      bestLabel = classValue
  return bestLabel

#多重预测
def getPredictions(summaries, testSet):
  predictions = []
  for i in range(len(testSet)):
    #回到计算所属类的概率
    result = predict(summaries, testSet[i][1:])
    predictions.append(result)
  return predictions

#计算精度
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][0] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

def main():
    filename = 'wine.csv'
    splitRatio = 0.67
    dataset = loadCsv(filename)
    #划分数据集
    trainingSet, testSet = splitDataset(dataset, splitRatio)
    print('Split {0} rows into train={1} and test={2} rows'.format(len(dataset), len(trainingSet), len(testSet)))
     # prepare model
    #将训练集按类别提取均值和方差的特征
    summaries = summarizeByClass(trainingSet)
    # test model
    predictions = getPredictions(summaries, testSet)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: {0}%'.format(accuracy))
main()

Split 178 rows into train=119 and test=59 rows
Accuracy: 98.30508474576271%
